# Setup
Setting up connections to AI Studio

In [1]:
import json

# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


# Testing Contoso Chat w/ All Data
Running base flow

In [4]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
flow = "../contoso-chat"
data = "../data/alltestdata.jsonl"
run_name = "contoso-chat-all"
column_mapping = {"customerId": "${data.customerId}", "question": "${data.question}"}

In [5]:
base_run = run_azure_flow(
    runtime, flow, run_name, data, column_mapping, pf_azure_client
)
pf_azure_client.stream(base_run)

Uploading alltestdata.jsonl (< 1 MB): 100%|██████████| 1.51k/1.51k [00:00<00:00, 7.77kB/s]


[2024-03-04 19:37:58,903][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/contoso-chat-all_03_04_1937/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
2024-03-04 11:38:08 +0000     112 promptflow-runtime INFO     [contoso-chat-all_03_04_1937] Receiving v2 bulk run request 706f0f0b-5a76-4463-82d1-7b2c81c3e2f6: {"flow_id": "contoso-chat-all_03_04_1937", "flow_run_id": "contoso-chat-all_03_04_1937", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "9ab9548d-1e28-41af-bca9-851af3611991"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontosoy46u2sqpv3h6g.blob.core.windows.net/8927edb2-b165-4c51-a19d-7201534af4ac-azureml/ExperimentRun/dcid.contoso-chat-all_03_04_1937/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=3d6e6777-5406-46a6-9b64-dbf0f2b204d3&sktid=16b3c013-d300-468d-ac64-7eda0

In [6]:
details = pf_azure_client.get_details(base_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
outputs.line_number,,,,,,
0,4,tell me about your hiking jackets,0,[],Hey Sarah Lee! 😊 Let me tell you about our awe...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hey John! 👋 Absolutely, we have some awesome c...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],Hey Michael! 👋 We have a great selection of te...,"[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],Hey Emily! 👋 We have some awesome hiking boots...,"[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hey Jane Doe! 🌲🥾 For hiking gear, I highly rec...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."
5,7,what is the temperature rating of my sleeping ...,5,[],Hey Jason Brown! 🌟 The temperature rating of y...,"[{'id': '7', 'title': 'CozyNights Sleeping Bag..."
6,7,what is the temperature rating of the cozynigh...,6,[],Hey Jason! 👋 The CozyNights Sleeping Bag has a...,"[{'id': '7', 'title': 'CozyNights Sleeping Bag..."
7,8,what is the waterproof rating of the tent I bo...,7,[],Hey Melissa! 🌟 The TrailMaster X4 Tent you bou...,"[{'id': '8', 'title': 'Alpine Explorer Tent', ..."
8,8,what is the waterproof rating of the TrailMast...,8,[],Hey Melissa! The rainfly of the TrailMaster X4...,"[{'id': '1', 'title': 'TrailMaster X4 Tent', '..."


Running Evaluation

In [7]:
runtime = "automatic"
eval_flow = "multi_flow/"
data = "../data/alltestdata.jsonl"
run_name = "contoso-chat-all-eval"

column_mapping = {
    # reference data
    "customerId": "${data.customerId}",
    "question": "${data.question}",
    "context": "${run.outputs.context}",
    # reference the run's output
    "answer": "${run.outputs.answer}",
}

In [8]:
eval_run = run_azure_eval_flow(
    runtime, eval_flow, run_name, data, column_mapping, base_run, pf_azure_client
)
pf_azure_client.stream(eval_run)

[2024-03-04 19:39:16,863][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/contoso-chat-all-eval_03_04_1939/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
2024-03-04 11:39:25 +0000      50 promptflow-runtime INFO     [contoso-chat-all-eval_03_04_1939] Receiving v2 bulk run request 6a9c3a36-e727-49be-a4c1-41b549d8d991: {"flow_id": "contoso-chat-all-eval_03_04_1939", "flow_run_id": "contoso-chat-all-eval_03_04_1939", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "c47343e5-115b-42e1-af06-b6e9c1202eeb"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://stcontosoy46u2sqpv3h6g.blob.core.windows.net/8927edb2-b165-4c51-a19d-7201534af4ac-azureml/ExperimentRun/dcid.contoso-chat-all-eval_03_04_1939/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b&sig=**data_scrubbed**&skoid=3d6e6777-5406-46a6-9b64-dbf0f2b204d3&sktid=16b3

In [9]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

,inputs.customerId,inputs.question,inputs.context,inputs.answer,inputs.line_number,inputs.chat_history,outputs.gpt_coherence,outputs.gpt_fluency,outputs.gpt_groundedness,outputs.gpt_relevance
outputs.line_number,,,,,,,,,,
0,4,tell me about your hiking jackets,"[{'id': '17', 'title': 'RainGuard Hiking Jacke...",Hey Sarah Lee! 😊 Let me tell you about our awe...,0,[],5.0,5.0,5.0,5.0
1,1,Do you have any climbing gear?,"[{'id': '9', 'title': 'SummitClimber Backpack'...","Hey John! 👋 Absolutely, we have some awesome c...",1,[],5.0,5.0,5.0,5.0
2,3,Can you tell me about your selection of tents?,"[{'id': '15', 'title': 'SkyView 2-Person Tent'...",Hey Michael! 👋 We have a great selection of te...,2,[],5.0,5.0,5.0,5.0
3,6,Do you have any hiking boots?,"[{'id': '4', 'title': 'TrekReady Hiking Boots'...",Hey Emily! 👋 We have some awesome hiking boots...,3,[],5.0,5.0,5.0,5.0
4,2,What gear do you recommend for hiking?,"[{'id': '10', 'title': 'TrailBlaze Hiking Pant...","Hey Jane Doe! 🌲🥾 For hiking gear, I highly rec...",4,[],4.0,5.0,5.0,5.0
5,7,what is the temperature rating of my sleeping ...,"[{'id': '7', 'title': 'CozyNights Sleeping Bag...",Hey Jason Brown! 🌟 The temperature rating of y...,5,[],4.0,4.0,3.0,3.0
6,7,what is the temperature rating of the cozynigh...,"[{'id': '7', 'title': 'CozyNights Sleeping Bag...",Hey Jason! 👋 The CozyNights Sleeping Bag has a...,6,[],3.0,3.0,3.0,2.0
7,8,what is the waterproof rating of the tent I bo...,"[{'id': '8', 'title': 'Alpine Explorer Tent', ...",Hey Melissa! 🌟 The TrailMaster X4 Tent you bou...,7,[],4.0,5.0,3.0,3.0
8,8,what is the waterproof rating of the TrailMast...,"[{'id': '1', 'title': 'TrailMaster X4 Tent', '...",Hey Melissa! The rainfly of the TrailMaster X4...,8,[],4.0,4.0,3.0,3.0


In [10]:
metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

{
    "gpt_coherence": 4.42,
    "gpt_coherence_pass_rate(%)": 91.67,
    "gpt_fluency": 4.67,
    "gpt_fluency_pass_rate(%)": 91.67,
    "gpt_relevance": 3.67,
    "gpt_relevance_pass_rate(%)": 50.0,
    "gpt_groundedness": 3.5,
    "gpt_groundedness_pass_rate(%)": 41.67
}


In [11]:
pf_azure_client.visualize([base_run, eval_run])

Web View: https://ml.azure.com/prompts/flow/bulkrun/runs/outputs?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourceGroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj&runId=contoso-chat-all_03_04_1937,contoso-chat-all-eval_03_04_1939
